In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
from sklearn.pipeline import Pipeline
from tensorflow import keras
import cv2

## YOLO Model for face detection

In [ ]:
%pip install ultralytics
from ultralytics import YOLO

#Load the pre-trained YOLOV8 model
path="/content/drive/MyDrive/Colab_Notebooks/runs/detect/train/weights/best.pt"
yolo_model = YOLO(path)

import numpy as np
import pandas as pd
from PIL import Image

def crop_face(image, model):
    """
    takes the image and the model, return a cropped cow face
    """
    # Convert the input image to a PIL Image object if it is not already
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    # Ensure the image is in RGB format
    image_rgb = image.convert('RGB')

    results = model(np.array(image_rgb), conf = 0.5)  # predict on an image
    a = results[0].boxes.boxes
    px = pd.DataFrame(a).astype("float")

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])

        if (x1 != 0) and (x2 != 0):
            # Crop the object from the image
            object_image = image_rgb.crop((x1, y1, x2, y2))

            # Resize the image to the desired shape (224, 224)
            resized_image = object_image.resize((224, 224), Image.ANTIALIAS)

            return resized_image

        else:
            return None


##VGG16 Classification model

In [5]:
class_names = ['cow1', 'cow10', 'cow11', 'cow12', 'cow13', 'cow14', 'cow15',
              'cow16', 'cow17', 'cow18', 'cow19', 'cow2', 'cow20', 'cow21',
              'cow22', 'cow3', 'cow4', 'cow5', 'cow6', 'cow7', 'cow8', 'cow9']
def predict_cow(cow_img, vgg16_model):
  """
  takes the image and the model, return the cow class
  """
  #predict on the image using saved model
  image=np.expand_dims(cow_img,axis=0)
  pred=vgg16_model.predict(image)
  output_class=class_names[np.argmax(pred)]
  return output_class



##Pipelining the models

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class CowFaceCropper(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        cropped_images = [crop_face(image, self.model) for image in X]
        return cropped_images


In [9]:
class CowClassPredictor(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        predictions = [predict_cow(image, self.model) for image in X]
        return predictions


In [16]:
from sklearn.pipeline import Pipeline

#loading the vgg16_model
vgg16_model = keras.models.load_model('/content/drive/MyDrive/cow_id_AI/vgg16_62')

# Initialize the custom transformers with their respective models
face_cropper = CowFaceCropper(yolo_model)
class_predictor = CowClassPredictor(vgg16_model)

# Create the pipeline
pipeline = Pipeline([
    ('face_cropper', face_cropper),
    ('class_predictor', class_predictor)
])

# Use the pipeline to predict cow classes for a list of input images
image1 = cv2.imread("/content/drive/MyDrive/cow_id_AI/DATA/a_c113.jpg")
image2 = cv2.imread("/content/drive/MyDrive/cow_id_AI/DATA/a_c13.jpg")
image3 = cv2.imread("/content/drive/MyDrive/cow_id_AI/DATA/a_c37.jpg")
image4 = cv2.imread("/content/drive/MyDrive/cow_id_AI/DATA/a_c12.jpg")
input_images = [image1, image2, image4]  # List of input images
predictions = pipeline.transform(input_images)



0: 640x384 1 Cow Face, 120.0ms
Speed: 0.8ms preprocess, 120.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x384 1 Cow Face, 119.6ms
Speed: 1.5ms preprocess, 119.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x384 1 Cow Face, 115.4ms
Speed: 1.2ms preprocess, 115.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


1/1 [==============================] - 1s 693ms/step
The predicted class is cow5
1/1 [==============================] - 1s 545ms/step
The predicted class is cow7
1/1 [==============================] - 1s 568ms/step
The predicted class is cow22
